In [24]:
from instaloader import Instaloader, Profile
import json

In [150]:
# Testate giornalistiche più importanti per paese da cui sono stati ricavati i post
lista_profili = ["lemondefr", "elmundo_es","corriere","nzz","nytimes","globeandmail","ukr_pravda","izvestia_ru"]

In [ ]:
L = Instaloader()
L.load_session_from_file("USERNAME") 
#creazione della sessione 
#prima di avviare scrive su cmd:istaloader --login USERNAME
#dopo si inserisce la password

In [35]:
# chiave dizionario profilo, valore lista di post
dictPost = {}

In [37]:
#scraper dei posti di "Aprile"
for profilo in lista_profili:
    profile_content = Profile.from_username(L.context, profilo)
    listaPost=[]
    for post in profile_content.get_posts():
        if post.date.month > 4:
            continue
        if post.date.month < 4:
            break
        listaPost.append(post)
    dictPost[profilo] = listaPost

In [48]:
#Ordino i post in ordine decrescente per numero_commenti+numero_likes, per prendere i post più significativi
dictPost2={profilo:sorted(lista, key=lambda x: x.likes+x.comments, reverse=True) for profilo, lista in dictPost.items()}

In [59]:
#Mi prendo i 5 post più significativi
dictPost3={profilo:lista[:5] for profilo, lista in dictPost2.items()}

In [ ]:
#Mi prendo tutte le informazioni del post e dei commenti associati al post
dictPost4={}
n_limit=298
for profilo,posts in dictPost3.items():
    dictPost4[profilo]=[]
    for post in posts:
        postSistemato={
        "id":post.shortcode,
        "date":str(post.date),
        "caption":post.caption,
        "caption_mentions":post.caption_mentions,
        "captions_hashtags":post.caption_hashtags,
        "n_comments":post.comments,
        "n_likes":post.likes,
        "comments":[]        
        }
        i=0
        for comment in post.get_comments():
            commentSistemato={
                "id":comment.id,
                "date":str(comment.created_at_utc),
                "text":comment.text,
                "likes":comment.likes_count,
                "owner":comment.owner.username,
            }
            postSistemato["comments"].append(commentSistemato)
            i+=1
            if i>n_limit:
                break
        dictPost4[profilo].append(postSistemato)

In [159]:
#Salvo i dati in un file json
json.dump(dictPost4, open("dictCompleto.json", "w"))